## 5.6 Affine / Softmaxレイヤの実装

### 5.6.1 Affineレイヤ ～ 5.6.2 バッチ版Affineレイヤの復習
<font size=4>■Affineとは？</font>
<br>ニューラルネットワークの順伝播で行う行列の内積処理は、幾何学の分野では「アフィン変換」と呼ばれる。
<br>このため、本書ではアフィン変換を行う処理を「Affineレイヤ」という名前で呼び、実装します。

<font size=4>■ニューラルネットの内積処理（順伝播の流れ）とは？</font>
<br>以下のような処理を行うこと。

In [8]:
import numpy as np
#入力
X = np.random.rand(2) #1行2列の行列（ベクトル）を作る
#重み
W = np.random.rand(2,3)#2行3列の行列を作る
#バイアス
B = np.random.rand(3) #1行3列の行列（ベクトル）を作る
print("X:")
print(X)
print("W:")
print(W)
print("B:")
print(B)

#XとWの内積を行い、バイアスを加算
##イメージはP58 図3-14にバイアスを加算したもの
Y = np.dot(X,W) + B
print("Y:")
print(Y)


X:
[ 0.40793469  0.49687236]
W:
[[ 0.24357109  0.32002027  0.6062862 ]
 [ 0.21971543  0.1130781   0.24374492]]
B:
[ 0.8457568   0.20431532  0.42363894]
Y:
[ 1.05428842  0.39104807  0.79207423]


このとき注意すべきことは、行列の対応する次元数を一致させることです。
<br><br>
![5-18内積注意点](".\img\5-18内積注意点.png")
<img src = ".\img\5-18内積注意点.png" width="400" height="300">
<br><br>
これまでの順伝播（内積処理）を計算グラフで記載すると。
<br>なお、各変数の上部に変数の形状も表記。
<br><br>
![5-19Affineレイヤ計算グラフ](".\img\5-19Affineレイヤ計算グラフ.png")
<img src = ".\img\5-19Affineレイヤ計算グラフ.png" width="400" height="300">
<br><br>
これまでの計算グラフは、「スカラ値」がノード間をながれていましたが、本例では「行列」がノード間を伝播しています。
<br><br>
<font size=4>■Affineの逆伝播処理</font>
<br><br>
![5-20Affineレイヤ逆伝播](".\img\5-20Affineレイヤ逆伝播.png")
<img src = ".\img\5-20Affineレイヤ逆伝播.png" width="600" height="500">
<br><br>
書籍ではいきなり
$$\frac{ \partial L }{ \partial X } = \frac{ \partial L }{ \partial Y } ・ W^{ \mathrm{ T } }$$
$$\frac{ \partial L }{ \partial W } = X^{ \mathrm{ T } } ・ \frac{ \partial L }{ \partial Y } $$
と出てくる。
<br>これは意味不明だと思うので、次の例で確かめてみる。

<br><br>
![5-22Affineレイヤ逆伝播お勉強](".\img\5-22Affineレイヤ逆伝播お勉強.png")
<img src = ".\img\5-22Affineレイヤ逆伝播お勉強.png" width="500" height="400">
<br><br>

入力として$x=(x_1,x_2)$の２次元だとします。
この入力に対して、第１層目への出力を３つにしたい場合、(2,3)の行列を右からかけます。つまり重みWを
$$W = \begin{pmatrix} w_{11} & w_{21} & w_{31} \\ w_{12} & w_{22} & w_{32} \end{pmatrix}$$
とします。つまり、出力Ｙは、
$$Y = X・W$$
$$  = \begin{pmatrix} x_1 & x_2 \end{pmatrix} ・ \begin{pmatrix} w_{11} & w_{21} & w_{31} \\ w_{12} & w_{22} & w_{32} \end{pmatrix}$$
$$  = \begin{pmatrix} w_{11}・x_1+w_{12}・x_2 & w_{21}・x_1+w_{22}・x_2 & w_{31}・x_1+w_{32}・x_2 \end{pmatrix} $$
$$  = \begin{pmatrix} y_1 & y_2 & y_3 \end{pmatrix} $$
となります。
<br>損失関数LLの入力Xによる偏微分は$x_1,x_2がy_1,y_2,y_3$に出てくることに注意すると、
$$\frac{ \partial L }{ \partial X } = \begin{pmatrix} \frac{ \partial L }{ \partial x_1 } & \frac{ \partial L }{ \partial x_2 } \end{pmatrix}$$
$$= \begin{pmatrix} \frac{ \partial L }{ \partial Y }・\frac{ \partial Y }{ \partial x_1 } & \frac{ \partial L }{ \partial Y }・\frac{ \partial Y }{ \partial x_2 } \end{pmatrix}$$
ここで、
$$\frac{ \partial L }{ \partial Y }・\frac{ \partial Y }{ \partial x_1 }  = \begin{pmatrix} \frac{ \partial L }{ \partial y_1 } & \frac{ \partial L }{ \partial y_2 } & \frac{ \partial L }{ \partial y_2 } \end{pmatrix}・\begin{pmatrix} \frac{ \partial y_1 }{ \partial x_1 } \\ \frac{ \partial y_2 }{ \partial x_2 } \\ \frac{ \partial y_3 }{ \partial x_3 } \end{pmatrix}$$
なので、
$$\frac{ \partial L }{ \partial X } =  \begin{pmatrix} \frac{ \partial L }{ \partial y_1 }・\frac{ \partial y_1 }{ \partial x_1 }+\frac{ \partial L }{ \partial y_2 }・\frac{ \partial y_2 }{ \partial x_1 } + \frac{ \partial L }{ \partial y_3 }・\frac{ \partial y_3 }{ \partial x_1 } & \frac{ \partial L }{ \partial y_1 }・\frac{ \partial y_1 }{ \partial x_2 }+\frac{ \partial L }{ \partial y_2 }・\frac{ \partial y_2 }{ \partial x_2 } + \frac{ \partial L }{ \partial y_3 }・\frac{ \partial y_3 }{ \partial x_2 } \end{pmatrix}$$
$$ = \begin{pmatrix} \frac{ \partial L }{ \partial y_1 }・w_{11} + \frac{ \partial L }{ \partial y_2 }・w_{21} + \frac{ \partial L }{ \partial y_3 }・w_{31} & \frac{ \partial L }{ \partial y_1 }・w_{12} + \frac{ \partial L }{ \partial y_2 }・w_{22} + \frac{ \partial L }{ \partial y_3 }・w_{32} \end{pmatrix}$$
$$ = \begin{pmatrix} \frac{ \partial L }{ \partial y_1 } & \frac{ \partial L }{ \partial y_2 } & \frac{ \partial L }{ \partial y_2 } \end{pmatrix}・ \begin{pmatrix} w_{11} & w_{12} \\ w_{21} & w_{22} \\ w_{31} & w_{32} \end{pmatrix}$$
$$ = \frac{ \partial L }{ \partial Y }・W^{ \mathrm{ T } }$$
<br>
<br>※参考サイト
<br>Affineレイヤの逆伝播を地道に成分計算する
<br>https://qiita.com/yuyasat/items/d9cdd4401221df5375b6


### 5.6.2 バッチ版Affineレイヤの実装

これまでの説明は入力であるXは一つのデータ（ベクトル）を対象としたものでしたが、ここではバッチ版のAffineレイヤとしてＮ個のデータをまとめて（行列化して）順伝播する方法を考えます。バッチ版のAffineレイヤは次のようになります。
<br><br>
![5-21バッチ版Affineレイヤ](".\img\5-21バッチ版Affineレイヤ.png")
<img src = ".\img\5-21バッチ版Affineレイヤ.png" width="600" height="500">
<br><br>
先ほどまでと異なる点は、入力であるXの形状が(N, 2)になっただけです。後は、先ほどまでと同様に計算グラフ上で計算することができます。また、逆伝播も行列の形状にだけ注意すれば、$\frac{ \partial L }{ \partial X }$、$\frac{ \partial L }{ \partial W }$も同様に導出できます。
<br>ただし、バイアスの計算は注意が必要です。順伝播の加算はバイアスがそれぞれのデータに加算されます。

In [3]:
import numpy as np
x_dot_w = np.array([[0,0,0],[10,10,10]])
b = np.array([1,2,3])
print("x_dot_w")
print(x_dot_w)
print("x_dot_w + b")
print(x_dot_w + b)


x_dot_w
[[ 0  0  0]
 [10 10 10]]
x_dot_w + b
[[ 1  2  3]
 [11 12 13]]


In [7]:
import numpy as np
#入力
X_dot_W = np.array([[0,0,0],[10,10,10]]) 
#バイアス
B = np.array([1,2,3]) 
print("X_dot_W:")
print(X_dot_W)
print("B:")
print(B)
print("X_dot_W + B:")
print(X_dot_W + B)

#XとWの内積を行い、バイアスを加算
#Y = np.dot(X,W) + B
#print("Y:")
#print(Y)


X_dot_W:
[[ 0  0  0]
 [10 10 10]]
B:
[1 2 3]
X_dot_W + B:
[[ 1  2  3]
 [11 12 13]]


このため、逆伝播の際には、それぞれのデータの逆伝播の値がバイアスの要素に集約される必要があります。

In [4]:
import numpy as np
dY = np.array([[1,2,3],[4,5,6]])
print("dY")
print(dY)
dB = np.sum(dY, axis=0)
print("dB")
print(dB)

dY
[[1 2 3]
 [4 5 6]]
dB
[5 7 9]


この例では、データが2個（N=2)あるものと仮定します。バイアスの逆伝播は、<b>その2個のデータに対しての微分を、データごとに合算して求めます(？)。</b><b>そのため、np.sum()で0番目の軸に対して総和を求めるのです(？)。</b>以上からAffineレイヤの実装は以下の通りとなります。

In [ ]:
Class Affine:
    #インスタンス生成時の処理
    def __init__(self, W ,b):
        self.W = W
        self.b = b
        self.x = None
        self.dw = None
        self.db = None
    
    #順伝播
    def forward(self,x):
        self.x = x #自身のXにXを代入
        out = np.dot(x, self.W) + self.b #内積計算+バイアス
        
        return out
    
    #逆伝播
    def backward(self, dout):
        dx = np.dot(dout, self.W.T) #X側の逆伝播を求める dout:前の層から逆伝播されてきた値と重みWの転置行列の内積
        self.dw = np.dot(self.x.T, dout) #W側の逆伝播を求める dout:前の層から逆伝播されてきた値と入力値Xの転置行列の内積
        self.db = np.sum(dout, axis=0) #データごとに合計していることは分るが、なんで逆伝播がこれでOKかは不明。。
        
        return dx
        

### 5.6.3　Softmax-with-Lossレイヤ
最後に、出力層であるソフトマックス関数について説明します。ソフトマックス関数は、入力された値を正規化して出力します。
<br><br>
![5-23SoftMax例.png](".\img\5-23SoftMax例.png")
<img src = ".\img\5-23SoftMax例.png" width="500" height="400">
<br><br>
本書では、Softmaxレイヤの実装に交差エントロピー誤差も含めて「Softmax-With-Lossレイヤ」という名前で実装しています。
それでは、このレイヤの計算グラフを示します。
<br><br>
![5-24SoftMax-With-Lossレイヤ.png](".\img\5-24SoftMax-With-Lossレイヤ.png")
<img src = ".\img\5-24SoftMax-With-Lossレイヤ.png" width="600" height="500">
<br><br>


### Softmax-With-Lossレイヤの計算グラフ
### ■順伝播
まずは、Softmaxレイヤです。Softmax関数は以下の数式で表されます。
$$  y_k = \frac{ \exp ( a_k ) }{ \displaystyle \sum_{ i = 1 }^{ n } \exp ( a_i ) } $$
これを計算グラフで表すと以下のようになります。
<br><br>
![5-26SoftMaxレイヤ順伝播.png](".\img\5-26SoftMaxレイヤ順伝播.png")
<img src = ".\img\5-26SoftMaxレイヤ順伝播.png" width="600" height="500">
<br><br>
続いて、Cross Entropy Errorです。Cross Entropy Errorは数式で次のようにあらわされます。
$$ L = -\sum_{k}^{ } t_k \log  y_k   $$
これを計算グラフで表すと以下のようになります。
<br><br>
![5-27CrossEntropyErrorレイヤ順伝播.png](".\img\5-27CrossEntropyErrorレイヤ順伝播.png)
<img src = ".\img\5-27CrossEntropyErrorレイヤ順伝播.png" width="600" height="500">
<br><br>
### ■逆伝播
まずは、Cross Entropy Errorの逆伝播を考えます。
以下のポイントを抑えれば、逆伝播は簡単です。
#### 1.『×』ノードの逆伝播は、順伝播時の入力値をひっくり返した値を、上流からの微分に乗算して下流に流す
#### 2.『＋』ノードでは、上流からの微分をそのまま流す
#### 3.『log』の逆伝播は次の式に従う

$$ y = \log x$$
$$ \frac{ \partial y }{ \partial x } = \frac{ 1 }{ x }$$ 
計算グラフで表すと以下のようになります。
<br><br>
![5-28CrossEntropyErrorレイヤ逆伝播.png](".\img\5-28CrossEntropyErrorレイヤ逆伝播.png)
<img src = ".\img\5-28CrossEntropyErrorレイヤ逆伝播.png" width="600" height="500">
<br><br>
次に、Softmaxレイヤの逆伝播を考えます。複雑なので、一つずつ確認しながら考えます。
<br><br>
![5-29SoftMaxレイヤ逆伝播1.png](".\img\5-29SoftMaxレイヤ逆伝播1.png)
<img src = ".\img\5-29SoftMaxレイヤ逆伝播1.png" width="600" height="500">
<br><br>
まず、前レイヤ（Cross Entropy Errorレイヤ）から逆伝播の値が流れてきます。
<br><br>
![5-30SoftMaxレイヤ逆伝播2.png](".\img\5-30SoftMaxレイヤ逆伝播2.png)
<img src = ".\img\5-30SoftMaxレイヤ逆伝播2.png" width="600" height="500">
<br><br>
「×」ノードでは順伝播の値をひっくり返して乗算します。ここでは次の計算が行われます。
$$ - \frac{ t_1 }{ y_1 } \exp(a_1) = -t_1 \frac{ S }{ \exp(a_1) } \exp(a_1) = -t_1 S $$
<br><br>
![5-31SoftMaxレイヤ逆伝播3.png](".\img\5-31SoftMaxレイヤ逆伝播3.png)
<img src = ".\img\5-31SoftMaxレイヤ逆伝播3.png" width="600" height="500">
<br><br>
順伝播で枝分かれした場合、逆伝播ではそれらの値が加算されます。つまり$(-t_1S,-t_2S,-t_3S)$が加算されます。そしてその加算された値に「/」の逆伝播を行うので、結果は1/s(t_1+t_2+t_3)となります（？）。またここでtはOne-hot表現で対応する値は1でそれ以外は0であるので、t_1～t_3の和は1となります。
<br><br>
![5-32SoftMaxレイヤ逆伝播4.png](".\img\5-32SoftMaxレイヤ逆伝播4.png)
<img src = ".\img\5-32SoftMaxレイヤ逆伝播4.png" width="600" height="500">
<br><br>
「＋」ノードは流すだけ。
<br><br>
![5-33SoftMaxレイヤ逆伝播5.png](".\img\5-33SoftMaxレイヤ逆伝播5.png)
<img src = ".\img\5-33SoftMaxレイヤ逆伝播5.png" width="600" height="500">
<br><br>
「×」ノードはひっくり返して乗算。
<br><br>
![5-34SoftMaxレイヤ逆伝播6.png](".\img\5-34SoftMaxレイヤ逆伝播6.png)
<img src = ".\img\5-34SoftMaxレイヤ逆伝播6.png" width="600" height="500">
<br><br>



上記の計算グラフを簡略化して書くと以下のようにできます。
<br><br>
![5-25SoftMax-With-Lossレイヤ簡略版.png](".\img\5-25SoftMax-With-Lossレイヤ簡略版.png")
<img src = ".\img\5-25SoftMax-With-Lossレイヤ簡略版.png" width="600" height="500">
<br><br>
ここでは、3クラス分類を行う場合を想定し、前レイヤから３つの入力（スコア）を受け取るものとします。Softmaxレイヤは、入力である$\begin{pmatrix} a_1 & a_2 & a_3 \end{pmatrix}$を正規化して、$\begin{pmatrix} y_1 & y_2 & y_3 \end{pmatrix}$を出力します。Corss Entropy Errorレイヤは、Softmaxの出力$\begin{pmatrix} y_1 & y_2 & y_3 \end{pmatrix}$と、教師ラベルの$\begin{pmatrix} t_1 & t_2 & t_3 \end{pmatrix}$を受け取り、それらのデータから損失Lを出力します。
<br>ここで注目すべきは、逆伝播の結果です。Softmaxレイヤからの逆伝播は、$\begin{pmatrix} y_1 - t_1 & y_2 - t_2 & y_3  - t_3\end{pmatrix}$というSoftmaxレイヤの出力と教師データの差分という<b>『キレイ』</b>な結果になっています。ニューラルネットワークの逆伝播では、この差分である誤差が前レイヤに伝わっていくのです。<b>この性質はニューラルネットの学習における重要な性質です。</b>
<br>このようなキレイな結果が返ってくるのは偶然でしょうか？これは偶然ではなく、このような結果が返ってくるように設計された関数が交差エントロピー誤差なのです。また、回帰問題では出力層に「恒等関数」を用いて、損失関数「2乗和誤差」を用いますが、これも同様の理由です。つまり、恒等関数の損失関数として2常和誤差を用いると、逆伝播が$\begin{pmatrix} y_1 - t_1 & y_2 - t_2 & y_3  - t_3\end{pmatrix}$というキレイな結果になります。
<br><br>
ここで具体例を考えてみましょう。
<br>例えば教師ラベルが$\begin{pmatrix} 0 & 1 & 0\end{pmatrix}$であるデータに対して、Softmaxレイヤの出力が$\begin{pmatrix} 0.3 & 0.2 & 0.5\end{pmatrix}$であった場合を考えます。正解ラベルに対する確率は20%(0.2)なので、この時点のニューラルネットワークは正しい認識ができていません。この場合、Softmaxレイヤからの逆伝播は、$\begin{pmatrix} 0.3 & -0.8 & 0.5\end{pmatrix}$という大きな誤差を伝播することとなりますので、前のレイヤではその大きな誤差から大きな内容を学習することになります。
<br>また、別の例として、教師ラベルが$\begin{pmatrix} 0 & 1 & 0\end{pmatrix}$であるデータに対して、Softmaxレイヤの出力が$\begin{pmatrix} 0.01 & 0.99 & 0\end{pmatrix}$であった場合を考えます。この場合Softmaxレイヤからの逆伝播の値は$\begin{pmatrix} 0.01 & -0.01 & 0\end{pmatrix}$という非常に小さい誤差になります。そのため、Softmaxレイヤより前のレイヤでは学習する内容も小さくなります。
それでは、Softmax with　Lossレイヤを実装します。

In [ ]:
class SoftmaxWithLoss:
    def ___init__(self):
        self.loss = None #損失
        self.y = None #Softmaxの出力
        self.t = None #教師データ(One-hot表現)
        
    def forward(self,x,t):
        self.t = t
        #過去に作ったSoftmax関数とCross Entropy Error関数を利用
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        
        return self.loss
        
    def backward(self, dout=1):
        #入力されたデータの行数（バッチの個数）を取得
        batch_size = self.shape[0]
        #バッチの個数で割ることで、データ1個当たりの誤差が前レイヤに伝播するようにしている（？）
        dx = (self.y - self.t) / bitch_size
        
        return dx

In [5]:
import numpy as np

print("#####################　順伝播　###################")
#入力
X = np.random.rand(2) #1行2列の行列（ベクトル）を作る
#重み
W = np.random.rand(2,3) #2行3列の行列を作る
#バイアス
B = np.random.rand(3) #1行3列の行列（ベクトル）を作る
print("X:")
print(X)
print("W:")
print(W)
print("B:")
print(B)

#XとWの内積を行い、バイアスを加算
Y = np.dot(X,W) + B
print("Y:")
print(Y)
    
print("#####################　逆伝播　###################")  
Y_Bak = np.array([1,1,1])
##転置
X_t = X.reshape(-1,1) #1次元は普通に転置できないのでreshapeを使う。(-1,1)を指定することで縦横ベクトルを入れ替える
W_t = W.T #2次元以上の配列は、配列.Tで転置可能
print("X_t:")
print(X_t)
print("W_t:")
print(W_t)

##内積
result_dotX = np.dot(Y_Bak,W_t)
#result_dotW = np.dot(X_t,Y)
print("X:")
print(result_dotX)
#print("W:")
#print(result_dotW)
    

#####################　順伝播　###################
X:
[ 0.90371264  0.49823441]
W:
[[ 0.51374329  0.13308606  0.48219582]
 [ 0.60324287  0.69663883  0.82085857]]
B:
[ 0.10743976  0.15812806  0.96268844]
Y:
[ 0.87227242  0.62548905  1.80743489]
#####################　逆伝播　###################
X_t:
[[ 0.90371264]
 [ 0.49823441]]
W_t:
[[ 0.51374329  0.60324287]
 [ 0.13308606  0.69663883]
 [ 0.48219582  0.82085857]]
X:
[ 1.12902517  2.12074027]
